# Data 620 - Project 2
Week 4 Project 2 - Summer 2021
Author: Layla Quinones
Group Members: Maliat Islam, & Karim Hammoud

## Assignment
<ol>
    <li>Identify a large 2-node network dataset—you can start with a dataset in a repository.  Your data should meet the criteria that it consists of ties between and not within two (or more) distinct groups.</li>
    <li>Reduce the size of the network using a What can you infer about each of the distinct groups?method such as the island method described in chapter 4 of social network analysis.</li>
    <li>What can you infer about each of the distinct groups?</li>
</ol>

## Introduction
- What data did we choose and why? Where did you get it from? How was it gathered? Describe how it fullfills the requirements - Your data should meet the criteria that it consists of ties between and not within two (or more) distinct groups.

What are bipartite networks? Why are they important?
- nodes are divided into two sets
- nodes cannot connect to other nodes in its same set, it must connect only to a node in the other set

## Data Importing & Pre Processing

In [1]:
#Import libraries
import pandas as pd
import networkx as nx
import pylab as plt

#import data into dataframe from github link
rawData = pd.read_csv("https://raw.githubusercontent.com/akarimhammoud/DATA-620/main/Week_4_project_2/IMDB-Movie-Data.csv", sep =',') 
rawData.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


Describe the data - how many observations, what information is there.

We are interested in creating a 2-Mode network with actors and movies as nodes.

In [10]:
#dataframe with movie and actors only
movieActorRaw = rawData[['Title','Actors']]
movieActorRaw.head()

,Title,Actors
0,Guardians of the Galaxy,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S..."
1,Prometheus,"Noomi Rapace, Logan Marshall-Green, Michael Fa..."
2,Split,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar..."
3,Sing,"Matthew McConaughey,Reese Witherspoon, Seth Ma..."
4,Suicide Squad,"Will Smith, Jared Leto, Margot Robbie, Viola D..."
...,...,...
995,Secret in Their Eyes,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts..."
996,Hostel: Part II,"Lauren German, Heather Matarazzo, Bijou Philli..."
997,Step Up 2: The Streets,"Robert Hoffman, Briana Evigan, Cassie Ventura,..."
998,Search Party,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh..."


In [132]:
#seperate the Actors column and Title column
actors = list(rawData.Actors)
movies = list(rawData.Title)

In [133]:
#split each element in actors into a list
for i in range(len(actors)):
    actors[i] = actors[i].split(',')

In [134]:
#create a dictionary with movies and actors
dictionary = dict(zip(movies, actors))

In [135]:
#create a edgelist out of the dictionary
#create a DF from dict
imdbDF = pd.DataFrame.from_dict(dictionary, orient = 'index')
#create new DF with dict keys as rows and values as columns
imdbDF = pd.DataFrame(list(zip(list(imdbDF.T.keys()), list(imdbDF[0]), list(imdbDF[1]), list(imdbDF[2]), list(imdbDF[3]))), columns = ['Title', 'One', 'Two', 'Three', 'Four'])
#Pivot from wide to long format (With titles repeating)
imdbDF = pd.melt(imdbDF, id_vars=['Title'], value_vars=['One', 'Two', 'Three', 'Four'])
imdbDF = imdbDF[['Title','value']]
imdbDF = imdbDF.rename(columns = {'value': 'Actor'})
imdbDF.head()

,Title,Actor
0,Guardians of the Galaxy,Chris Pratt
1,Prometheus,Noomi Rapace
2,Split,James McAvoy
3,Sing,Matthew McConaughey
4,Suicide Squad,Will Smith


## Network Creation

In [137]:
#generate a list with all unique actors 
uniqueActors = list(imdbDF.Actor.unique())
#generate a list of all unique movies
uniqueMovie = list(imdbDF.Title.unique())
#Turn edge list into tuple
records = imdbDF.to_records(index=False)
result = list(records)

In [138]:
#create a Graph object
G = nx.Graph()

#specify the two different kinds of nodes
G.add_nodes_from(uniqueMovie,node_type='Movie', bipartite=0)
G.add_nodes_from(uniqueActors,node_type='Actors', bipartite=1)

#add the edges to the graph
G.add_edges_from(result)

#display information to confirm
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 3392
Number of edges: 3996
Average degree:   2.3561


In [ ]:
#visualize
nx.draw(G)
plt.title('IMDB Network', fontsize = 30, color = 'blue')
plt.show()

In [ ]:
#create a Graph object (this is the exact same at the method above)
G2 = nx.from_pandas_edgelist(imdbDF, 'Title','Actor')
nx.draw(G2)
plt.title('IMDB Network', fontsize = 30, color = 'blue')
plt.show()

In [139]:
#display information to confirm
print(nx.info(G2))

Name: 
Type: Graph
Number of nodes: 3392
Number of edges: 3996
Average degree:   2.3561


## Analysis
As we can see from the above network, our IMDB 2-mode network is fairly saturated and we can see that 

## Island Method

Describe what you are doing (subsetting) how and why.

Link to data : https://www.kaggle.com/PromptCloudHQ/imdb-data